In [ ]:
!apt-get install psmisc
!git clone https://github.com/kietngt00/hmdb51-recognition.git
%cd hmdb51-recognition
!apt-get install unrar
!bash download_annotation.sh
!mv hmdb_labels.txt ./dataset
# !git checkout <YOUR BRANCH> # Checkout to your branch

In [ ]:
!wget http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar

!mkdir ./dataset/hmdb51
!mv hmdb51_org.rar dataset/hmdb51

%cd ./dataset/hmdb51
!unrar x ./hmdb51_org.rar
!rm ./hmdb51_org.rar
!for file in *.rar; do unrar x $file; done
!rm ./*.rar
%cd /kaggle/working/hmdb51-recognition

In [ ]:
!pip install pytorchvideo
!pip install einops
!pip install addict
!pip install numpy
!pip install tensorboard

In [ ]:
%cd hmdb51-recognition

In [ ]:
import argparse
from pathlib import Path
import numpy as np
import glob
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from data import HMDB51DataModule
from model.model import ModelInterface
from utils import *

In [ ]:
cfg = read_yaml('config.yaml')

cfg.stage = 'train'
cfg.ckp_path = '<checkpoint>'

cfg.Data.data_path = './hmdb51-recognition/dataset/annotations'     # Path to annotation file
cfg.Data.video_path_prefix = './hmdb51-recognition/dataset/hmdb51'  # Path to video folder
cfg.Data.num_workers = 2
cfg.Data.num_frames = 8

#---->Initialize seed
pl.seed_everything(cfg.General.seed)

#---->load loggers
logger = load_loggers(cfg)

#---->load callbacks
callbacks = load_callbacks(cfg)

#---->Define Data 
dm = HMDB51DataModule(cfg.Data)

#---->Define Model
model = ModelInterface(cfg.Model)

#---->Instantiate Trainer
trainer = Trainer(
    num_sanity_val_steps=0, 
    logger=logger,
    callbacks=callbacks,
    max_epochs= cfg.General.epochs,
    accelerator='gpu',
    deterministic=True,
    check_val_every_n_epoch=1,
    log_every_n_steps=1,
    # limit_train_batches=1,
    # limit_val_batches=2,
)

In [ ]:
# !fuser -v /dev/nvidia*

In [ ]:
# !kill -9 <GPU PID>

In [ ]:
# !nvidia-smi

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir logs --port=6008

In [ ]:
#---->train or test
if cfg.stage == 'train': 
    trainer.fit(model=model, datamodule=dm)
    trainer.test(datamodule=dm, ckpt_path='best')
elif cfg.stage == 'test': 
    model = ModelInterface.load_from_checkpoint(cfg.ckp_path)
    model.eval()
    trainer.test(model=model, datamodule=dm)